## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Particle Swarm para optimización del orden de las intervenciones quirúrgicas

#### Autor: Jesús García Armario

In [79]:
# Imports necesarios
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico
from Genético.utilsGenetico import cromosomaAleatorio, evaluar, validar, distance, cruce, mutacion

In [80]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Sumamos a la duración 25 minutos por paciente para tener en cuenta el tiempo de preparación
df['DURACIÓN'] = df['DURACIÓN'] + 25
ventana = 30
# Dividimos la duración entre la ventana y redondeamos hacia arriba
df['DURACIÓN'] = df['DURACIÓN'].apply(lambda x: math.ceil(x/ventana))
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [81]:
# Creamos una población de 100 partículas válidas
def crear_poblacion(tiempos, quirofanos, dias, actos_pendientes, n):
    poblacion = []
    for i in range(n):
        poblacion.append(cromosomaAleatorio(tiempos, quirofanos, dias, actos_pendientes))
    return poblacion

In [82]:
# Iniciamos las velocidades de las partículas
def iniciar_velocidades(poblacion):
    pop = []
    for part in poblacion:
        particula = {}
        particula['cromosoma'] = part
        particula['velocidad'] = 0
        pop.append(particula)
    return pop

In [83]:

# Evaluamos a cada partícula
def evaluar_poblacion(poblacion, tiempos = 16, quirofanos = 3, dias = 5, actos_pendientes = actos_pendientes):
    for part in poblacion:
        part['fitness'] = evaluar(part['cromosoma'], tiempos, quirofanos, dias, actos_pendientes)
    return poblacion

# Inicializamos la mejor partícula
def mejor_particula(poblacion):
    mejor = poblacion[0]
    for part in poblacion:
        if part['fitness'] < mejor['fitness']:
            mejor = part
    return mejor

# Inicializamos la personal best de cada partícula
def personal_best(poblacion):
    for part in poblacion:
        part['personal_best'] = part['cromosoma']
        part['fitness_personal_best'] = part['fitness']
    return poblacion

In [84]:
# Creamos la población inicial
poblacion = crear_poblacion(16, 3, 5, actos_pendientes, 100)
# Iniciamos las velocidades
poblacion = iniciar_velocidades(poblacion)
# Evaluamos la población
poblacion = evaluar_poblacion(poblacion)
# Inicializamos la mejor partícula
mejor = mejor_particula(poblacion)
# Inicializamos la personal best de cada partícula
poblacion = personal_best(poblacion)
# Inicializamos los parámetros
w = 0.9
c1 = 0.5
c2 = 0.3
mejor_global = mejor
# Inicializamos el contador de iteraciones
iteraciones = 0
# Inicializamos el contador de iteraciones sin mejora
iteraciones_sin_mejora = 0
# Inicializamos el número máximo de iteraciones sin mejora
max_iteraciones_sin_mejora = 100
# Inicializamos el número máximo de iteraciones
max_iteraciones = 1000

print('Mejor solución inicial: ', mejor_global)

# Iniciamos el bucle
while iteraciones < max_iteraciones and iteraciones_sin_mejora < max_iteraciones_sin_mejora:
    # Actualizamos la mejor partícula global
    if mejor['fitness'][0] < mejor_global['fitness'][0]:
        mejor_global = mejor
        iteraciones_sin_mejora = 0
    else:
        iteraciones_sin_mejora += 1
    # Actualizamos las velocidades y posiciones
    for part in poblacion:
        # Actualizamos la velocidad
        part['velocidad'] = w * part['velocidad'] + c1 * random.random() * (mejor['fitness'][0] - part['fitness'][0]) + c2 * random.random() * (mejor_global['fitness'][0] - part['fitness'[0]])
        # Actualizamos la posición
        part['fitness'] = part['fitness'] + part['velocidad']
        # Realizamos la mutación
        part['cromosoma'] = mutacion(part['cromosoma'])
    # Seleccionamos los individuos a cruzar
    # Serán los mejores de la población
    poblacion = sorted(poblacion, key = lambda i: i['fitness'])
    # Cruzamos los dos mejores
    poblacion[0]['cromosoma'], poblacion[1]['cromosoma'] = cruce(poblacion[0]['cromosoma'], poblacion[1]['cromosoma'])
    # Evaluamos a los nuevos individuos
    poblacion[0]['fitness'] = evaluar(poblacion[0]['cromosoma'])
    poblacion[1]['fitness'] = evaluar(poblacion[1]['cromosoma'])
    # Actualizamos la personal best de cada partícula
    for part in poblacion:
        if part['fitness'] < part['fitness_personal_best']:
            part['personal_best'] = part['cromosoma']
            part['fitness_personal_best'] = part['fitness']

    # Actualizamos la mejor partícula
    mejor = mejor_particula(poblacion)
    # Actualizamos el contador de iteraciones
    iteraciones += 1

# Mostramos el resultado
print('Mejor solución: ', mejor_global['cromosoma'])
print('Fitness: ', mejor_global['fitness'])
print('Iteraciones: ', iteraciones)


Mejor solución inicial:  {'cromosoma': [59, 55, 97, 'A', 46, 43, 63, 62, 75, 'A', 13, 57, 95, 24, 93, 9, 77, 'A', 'B', 91, 40, 67, 14, 76, 'A', 1, 21, 99, 92, 23, 'V', 51, 'A', 74, 45, 27, 88, 0, 'A', 'B', 37, 96, 31, 38, 'V', 19, 'A', 86, 47, 87, 79, 22, 'A', 42, 72, 65, 18, 'A', 'B', 70, 58, 64, 16, 'A', 5, 15, 54, 29, 'A', 7, 60, 33, 17, 'A', 'B', 2, 4, 80, 83, 8, 'A', 73, 34, 56, 68, 'A', 12, 48, 35, 85, 30, 82, 'A', 'B'], 'velocidad': 0, 'fitness': (0.002335968479988713,), 'personal_best': [59, 55, 97, 'A', 46, 43, 63, 62, 75, 'A', 13, 57, 95, 24, 93, 9, 77, 'A', 'B', 91, 40, 67, 14, 76, 'A', 1, 21, 99, 92, 23, 'V', 51, 'A', 74, 45, 27, 88, 0, 'A', 'B', 37, 96, 31, 38, 'V', 19, 'A', 86, 47, 87, 79, 22, 'A', 42, 72, 65, 18, 'A', 'B', 70, 58, 64, 16, 'A', 5, 15, 54, 29, 'A', 7, 60, 33, 17, 'A', 'B', 2, 4, 80, 83, 8, 'A', 73, 34, 56, 68, 'A', 12, 48, 35, 85, 30, 82, 'A', 'B'], 'fitness_personal_best': (0.002335968479988713,)}


KeyError: 'f'